## 集成方法

- 概念: 是对其他算法进行组合的一种形式
	通俗来说，当做重要决定时，大家可能都会考虑吸取多个专家而不只是一个人的意见。 机器学习处理问题时又何尝不是如此？ 这就是集成方法背后的思想。
- 集成方法：
	
	投票选举(bagging: 自举汇聚法 bootstrap aggregating): 是基于数据随机重抽样分类器构造的方法

	再学习(boosting): 是基于所有分类器的加权求和的方法

### bossting和bagging的区别与联系
1. bagging 是一种与 boosting 很类似的技术, 所使用的多个分类器的类型（数据量和特征量）都是一致的。
2. bagging 是由不同的分类器（1.数据随机化 2.特征随机化）经过训练，综合得出的出现最多分类结果；boosting 是通过调整已有分类器错分的那些数据来获得新的分类器，得出目前最优的结果。
3. bagging 中的分类器权重是相等的；而 boosting 中的分类器加权求和，所以权重并不相等，每个权重代表的是其对应分类器在上一轮迭代中的成功度。

目前 bagging 方法最流行的版本是: 随机森林(random forest)。【投票选举，分高者得】

目前 boosting 方法最流行的版本是: AdaBoost。【不断优化，最优者得】

### AdaBoost
#### AdaBoost的一般流程
1. 收集数据：可以使用任意方法
2. 准备数据：依赖于所使用的弱分类器类型，本章使用的是单层决策树，这种分类器可以处理任何数据类型。
    当然也可以使用任意分类器作为弱分类器，第2章到第6章中的任一分类器都可以充当弱分类器。
    作为弱分类器，简单分类器的效果更好。
3. 分析数据：可以使用任意方法。
4. 训练算法：AdaBoost 的大部分时间都用在训练上，分类器将多次在同一数据集上训练弱分类器。
5. 测试算法：计算分类的错误率。
6. 使用算法：通SVM一样，AdaBoost 预测两个类别中的一个。如果想把它应用到多个类别的场景，那么就要像多类 SVM 中的做法一样对 AdaBoost 进行修改。

### AdaBoost算法特点
* 优点：泛化（由具体的、个别的扩大为一般的）错误率低，易编码，可以应用在大部分分类器上，无参数调节。
* 缺点：对离群点敏感。
* 适用数据类型：数值型和标称型数据。

### 单层决策树伪代码
    
    将最小错误率minError设置为+无穷
    对数据集中的每一个特征（第一层循环）:
        对每个步长（第二层循环）:
            对每个不等号（第三层循环）:
                建立一棵单层决策树并利用加权数据集对它进行测试
                如果错误率低于minError，则将当前单层决策树设置为最佳单层决策树
    返回最佳单层决策树

In [1]:
from numpy import *
from imp import *
import adaboost

In [2]:
datMat,classLabels = adaboost.loadSimpData()

In [3]:
D = mat(ones((5,1))/5)
adaboost.buildStump(datMat,classLabels,D)

({'dim': 0, 'ineq': 'lt', 'thresh': 1.3}, matrix([[ 0.2]]), array([[-1.],
        [ 1.],
        [-1.],
        [-1.],
        [ 1.]]))

### 完整AdaBoost算法的实现
伪代码：
```
对每次迭代：
    利用buidStrump()函数找到最佳的单层决策树
    将单层最佳决策树加入到单层决策树数组
    计算alpha
    计算新的权重向量D
    更新累计类别估计值
    如果错误率等于0.0，则退出循环
```

In [4]:
classifierArray = adaboost.adaBoostTrainDS(datMat,classLabels,9)

total error:  0.2
total error:  0.2
total error:  0.0


In [5]:
classifierArray

[{'alpha': 0.6931471805599453, 'dim': 0, 'ineq': 'lt', 'thresh': 1.3},
 {'alpha': 0.9729550745276565, 'dim': 1, 'ineq': 'lt', 'thresh': 1.0},
 {'alpha': 0.8958797346140273,
  'dim': 0,
  'ineq': 'lt',
  'thresh': 0.90000000000000002}]

### 测试算法：基于AdaBoost的分类
现在我们需要做的只是将弱分类器的训练过程从程序中抽出来，然后应用到某个具体的实例上去。

每个弱分类器的结果以及对应的alpha值作为权重，所有这些弱分类器的结果加权求和就得到了最后的结果。


In [6]:
adaboost.adaClassify([0,0],classifierArray)
#可以发现随着迭代的进行，数据点[0,0]的分类结果越来越强

[[-0.69314718]]
[[-1.66610226]]
[[-2.56198199]]


matrix([[-1.]])

### 示例：在一个难数据集上应用AdaBoost

1. 收集数据：提供的文本文件
2. 准备数据：确保类别标签是+1和-1而不是1和0
3. 分析数据：手工检查数据
4. 训练算法：在数据上，利用adaBoostTrainDS()函数训练处一系列的分类器
5. 测试算法：我们拥有两个数据集，在不采用随机抽样的方法下，我们就会对AdaBoost和Logistic回归的结果进行完全对等的比较。
6. 使用算法：观察该例子上的错误率。

In [148]:
reload(adaboost)
reload(adaboostDemo)
import adaboostDemo
datMat,labelMat = adaboostDemo.loadDataSet('horseColicTraining2.txt') 
classifierArray = adaboost.adaBoostTrainDS(datMat,labelMat,10)

299
total error:  0.284280936455
total error:  0.284280936455
total error:  0.247491638796
total error:  0.247491638796
total error:  0.254180602007
total error:  0.240802675585
total error:  0.240802675585
total error:  0.220735785953
total error:  0.247491638796
total error:  0.230769230769


In [155]:
reload(adaboostDemo)
testArr,testLabelArr = adaboostDemo.loadDataSet('horseColicTest2.txt')
testLabelMat = mat(testLabelArr)
prediction10 = adaboost.adaClassify(testArr,classifierArray)
errArr = mat(ones((67,1)))
errArr[prediction10!=testLabelMat.T].sum()

67


16.0

通过不断改变弱分类器的数目可以发现，不同分类器数目情况下的AdaBoost测试和分类错误率。该数据集是个难数据集。

通常情况下，AdaBoost会达到一个稳定的测试错误率，并不会随着分类器数目的增多而提高。